In [597]:
import sys  
import time  
import math  
import uuid  
import numpy as np  
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objs as go
from numpy import errstate  
from sklearn.datasets import fetch_openml  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import OneHotEncoder  

In [598]:
sys.setrecursionlimit(10000)  

In [599]:
class VarValue:
    def __init__(self, value, children=(), varname=''):
        self.varname = varname
        self.value = value
        self.derivative_to = {}
        self.children = children

    def __chain_rule(self, dSelfdx, child):
        if(child.varname[:5] != 'const'):
            for grandchild_varname in child.derivative_to:
                if(grandchild_varname != 'const'):
                    if(grandchild_varname in self.derivative_to):

                        self.derivative_to[grandchild_varname] += dSelfdx * child.derivative_to[grandchild_varname]
                    else:
                        self.derivative_to[grandchild_varname] = dSelfdx * child.derivative_to[grandchild_varname]
            if(len(self.derivative_to) == 0):
              raise ValueError(self.varname, child.varname)

    def relu(self):
        out = VarValue(max(0,self.value), children=(self,), varname='out_relu_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 0 if self.value <= 0 else 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def ln(self):
        out = VarValue(math.log(self.value), children=(self,), varname='out_ln_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 1/(self.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __mul__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value * other.value, children=(self, other), varname='out_mul_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx1 = other.value
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = self.value
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __add__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value + other.value, children=(self, other), varname='out_add_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx1 = 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = 1
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __pow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        try:
            with errstate(over='raise', invalid='raise'):
                result = float(self.value) ** float(other.value)
        except (OverflowError, FloatingPointError):
            if abs(float(self.value)) > 1:
                result = math.inf
            else:
                result = 0

        out = VarValue(result, children=(self, other), varname='out_pow_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx = other.value * self.value**(other.value-1)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return other + (-self)

    def __rpow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(other.value**self.value, children=(self, other), varname='out_rpow_'+str(uuid.uuid4()))
        if self.varname:
            dodx = other.value**self.value * math.log(other.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rtruediv__(self, other):
        return other * self**-1

    # Equality
    def __eq__(self, other):
        if isinstance(other, VarValue):
            return self.varname == other.varname
        return self.varname == other

    # Inequality
    def __ne__(self, other):
        return not self.__eq__(other)

    # Less than
    def __lt__(self, other):
        if isinstance(other, VarValue):
            return self.value < other.value
        return self.value < other

    # Less than or equal
    def __le__(self, other):
        if isinstance(other, VarValue):
            return self.value <= other.value
        return self.value <= other

    # Greater than
    def __gt__(self, other):
        if isinstance(other, VarValue):
            return self.value > other.value
        return self.value > other

    # Greater than or equal
    def __ge__(self, other):
        if isinstance(other, VarValue):
            return self.value >= other.value
        return self.value >= other

In [600]:
class Layer:
    def __init__(self,n_neurons=3, init='zero', activation='relu', weights=None, biases=None):
        # weights & biases ditambah di init buat memfasilitasi loading weight & bias
        self.n_neurons = n_neurons
        self.current_input_batch = None
        self.init = init    # zero/uniform/normal/xavier/he - harusnya gaperlu disini, ini di layer langsung harusnya
        self.weights = weights
        self.biases = biases
        self.activation = activation    # linear/relu/sigmoid/tanh/softmax/binary_step/leaky_relu/prelu/elu/swish(ini)/gelu(ini)
        self.learning_rate = None

        self.net = None
        self.out = None

    def __update_weights_dEdW(self, dEdW):
        self.weights -= self.learning_rate*dEdW

    def __update_biases_dEdB(self, dEdB):
        self.biases -= self.learning_rate*dEdB

    def __update_weights_err_term(self, err_term):
        for input in self.current_input_batch:
            for i in self.weights:
                for j in i:
                    self.weights += self.learning_rate*err_term[j]*self.input[i]

    def __update_biases_err_term(self, err_term):
        for i in self.weights:
            for j in i:
                self.weights += self.learning_rate*err_term[j]*1

    def forward(self, current_input_batch):
        self.current_input_batch = current_input_batch

        if(self.weights is None):
            
            if(self.init == 'zero'):
                self.weights = np.array([[VarValue(0,varname='w_'+str(uuid.uuid4())) for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(0,varname='b_'+str(uuid.uuid4())) for _ in range(self.n_neurons)])
                print("TESTT")
                print(self.weights)
                print(self.biases)

            # Ini semua diround soalnya hasil operasinya kegedean, kena warning wkwkwk. CMIIW yh harusnya berapa angka di belakang koma - @evelynnn04

            elif(self.init == 'uniform'):
                limit = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.uniform(-limit, limit), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.uniform(-limit, limit), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'normal':
                std = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.normal(0, std), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'xavier':
                std = np.sqrt(2 / (len(self.current_input_batch[0]) + self.n_neurons))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'he':
                std = np.sqrt(2 / len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])


        self.net = (np.dot(self.current_input_batch, self.weights)) + self.biases

        if(self.activation == 'linear'):
            self.out = self.net

        elif(self.activation == 'relu'):
            self.out = np.array([[net.relu() for net in row] for row in self.net])

        elif(self.activation == 'sigmoid'):
            for i in self.net:
                for j in i:
                    j.value = np.clip(j.value, -500, 500)
            self.out = 1 / (1 + (math.exp(1))**(-self.net))

        elif(self.activation == 'tanh'):
            self.out = (math.exp(1)**self.net - math.exp(1)**(-self.net))/(math.exp(1)**self.net + math.exp(1)**(-self.net))

    def backward(self, sum_err_term_k_w=None, err=None, method='instant_deriv'):
        
        if(method == 'instant_deriv'):

            dEdW = np.array([[ err.derivative_to[neuron_w.varname] for neuron_w in weights] for weights in self.weights])
            dEdB = np.array([ err.derivative_to[bias.varname] for bias in self.biases])

            self.__update_weights_dEdW(dEdW)
            self.__update_biases_dEdB(dEdB)

        else:
            # Versi pake Error Term - Not Tested Yet
            if(err):
                # Output Layer
                err_term = np.array([err.derivative_to[net.varname] for net in self.net])
            else:
                # Hidden Layer
                err_term = sum_err_term_k_w
                pass
            sum_err_term_w_current = self.weights @ err_term.T
            self.__update_weights_err_term(err_term)
            self.__update_biases_err_term(err_term)
            return sum_err_term_w_current,

    def clean_derivative(self):
        for input in self.current_input_batch:
            for x in input:
                x.derivative_to.clear()
                x.children = ()

        for i in self.weights:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for b in self.biases:
            b.derivative_to ={}
            b.children = ()

        for i in self.net:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for i in self.out:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

In [ ]:
class FFNN:
    def __init__(self, loss='mse', batch_size=1, learning_rate=0.01, epochs=20, verbose=0):
        self.loss = loss    # mse/bce/cce
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.verbose=verbose
        self.layers = None
        self.weights = []
        self.bias = []
        self.x = None
        self.y = None
        self.onehot_encoder = OneHotEncoder(categories='auto')

    def __loss(self, out, target):
        if(out.shape != target.shape):
            print("Output shape: ", out.shape)
            print("Target shape: ", target.shape)
            raise ValueError("Shape not match")
        if self.loss == 'mse':
            mse = (1/target.shape[1])*np.square(target - out)
            return np.sum(mse)
        elif self.loss == 'cce':
            epsilon = 1e-15
            out = np.clip(out, epsilon, 1 - epsilon)
            return -np.sum(target * np.log(out)) / self.batch_size

    def build_layers(self, *layers: Layer):
        self.layers = layers
        for layer in self.layers:
            layer.learning_rate = self.learning_rate

    def fit(self, x, y):
        self.x = x
        self.y = self.onehot_encoder.fit_transform(y.reshape(-1, 1))
        total_batch = (len(x)+self.batch_size-1)//self.batch_size
        start_global = time.time()

        if self.verbose == 1:
                epoch_iterator = tqdm(range(self.epochs), desc="Training Progress", unit="epoch")
        else:
            epoch_iterator = range(self.epochs)

        for epoch in epoch_iterator:
            print(f"Epoch {epoch+1}")
            start = time.time()
            # count_batch = 1

            if self.verbose == 1:
                batch_iterator = tqdm(range(total_batch), desc=f"Epoch {epoch + 1}", unit="batch", leave=False)
            else:
                batch_iterator = range(total_batch)
            
    
            for i in batch_iterator:
                x_batch = self.x[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.x[i*self.batch_size:]
                y_batch = self.y[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.y[i*self.batch_size:]
                batch_input = x_batch
                for layer in self.layers:
                    layer.forward(batch_input)
                    batch_input = layer.out
                out = batch_input
                error = self.__loss(out, y_batch)
                # count = 1

                for layer in self.layers[::-1]:
                    layer.backward(err=error)
                #     count += 1
                # count_batch += 1

                # Ini gaperlu harusnya gapapa
                for layer in self.layers:
                    layer.clean_derivative()

        end_global = time.time()
        print("Total Duration: ", end_global-start_global)

    def predict(self, x_predict):
        batch_input = x_predict
        for layer in self.layers:
            layer.forward(batch_input)
            batch_input = layer.out
        out = batch_input
        return out

    def visualize(self):
        num_layers = len(self.layers)
        
        # Color palette
        layer_colors = {
            0: 'yellow',    # Input layer
            -1: 'salmon',   # Output layer
            'hidden': 'lightblue'  # Hidden layers
        }
        
        nodes_x = []
        nodes_y = []
        node_colors = []
        node_texts = []
        
        edges_x = []
        edges_y = []
        edge_texts = []
        
        for layer_idx, layer in enumerate(self.layers):
            if num_layers == 1:
                color = layer_colors[-1]
            elif layer_idx == 0:
                color = layer_colors[0]
            elif layer_idx == num_layers - 1:
                color = layer_colors[-1]
            else:
                color = layer_colors['hidden']
            
            n_neurons = layer.n_neurons
            y_positions = np.linspace(0, 1, n_neurons)
            x_pos = layer_idx / (num_layers - 1) if num_layers > 1 else 0.5
            
            for neuron_idx, y_pos in enumerate(y_positions):
                nodes_x.append(x_pos)
                nodes_y.append(y_pos)
                node_colors.append(color)
                node_texts.append(f"Layer {layer_idx}, Neuron {neuron_idx}<br>Activation: {layer.activation}")
            
            if layer_idx < num_layers - 1:
                next_layer = self.layers[layer_idx + 1]
                next_x = (layer_idx + 1) / (num_layers - 1) if num_layers > 1 else 0.5
                next_n_neurons = next_layer.n_neurons
                next_y_positions = np.linspace(0, 1, next_n_neurons)
                
                for curr_neuron_idx, curr_y in enumerate(y_positions):
                    for next_neuron_idx, next_y in enumerate(next_y_positions):
 
                        edges_x.extend([x_pos, next_x, None])
                        edges_y.extend([curr_y, next_y, None])
                        
                        weight_text = "Weight: Not initialized"
                        if layer.weights is not None:
                            try:
                                weight = layer.weights[curr_neuron_idx][next_neuron_idx]
                                weight_text = f"Weight: {weight}"
                            except:
                                weight_text = "Weight: Unavailable"
                        
                        edge_texts.append(weight_text)
        
        # Create nodes
        node_trace = go.Scatter(
            x=nodes_x, y=nodes_y,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=False,
                color=node_colors,
                size=15,
                line_width=2
            ),
            text=node_texts
        )
        
        # Create edges
        edge_trace = go.Scatter(
            x=edges_x, y=edges_y,
            mode='lines',
            line=dict(width=0.5, color='#888'),
            hoverinfo='text',
            text=edge_texts
        )
        
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title='Neural Network Architecture',
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0,l=0,r=0,t=40),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                        ))
        
        fig.update_layout(plot_bgcolor='rgba(255,255,255,255)')
        fig.show()

    
    def save(self, filename):
        model_data = {
            'loss': self.loss,
            'batch_size': self.batch_size,
            'learning_rate': self.learning_rate,
            'epochs': self.epochs,
            'layers': [
                {'n_neurons': layer.n_neurons, 'activation': layer.activation, 'weights': layer.weights.tolist(), 'biases': layer.biases.tolist()}
                for layer in self.layers
            ]
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to {filename}")

    def load(self, filename):
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        self.loss = model_data['loss']
        self.batch_size = model_data['batch_size']
        self.learning_rate = model_data['learning_rate']
        self.epochs = model_data['epochs']
        self.layers = [
            Layer(n_neurons=layer['n_neurons'], activation=layer['activation'], weights=np.array(layer['weights']), biases=np.array(layer['biases']))
            for layer in model_data['layers']
        ]
        print(f"Model loaded from {filename}")


In [602]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = y.astype(np.uint8)

In [603]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=65500,
    train_size=10,
    random_state=42,
    stratify=y
)

In [604]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [605]:
X_train = np.array([[VarValue(x, varname='x_'+str(uuid.uuid4())) for x in row] for row in X_train])

In [606]:
model_ffnn = FFNN(
    loss='mse',
    batch_size=1,
    learning_rate=0.1,
    epochs=2,
    verbose=1,
)

In [607]:
model_ffnn.build_layers(
    Layer(n_neurons=4, init='uniform', activation='linear'),
    Layer(n_neurons=3, init='uniform', activation='linear'),
    Layer(n_neurons=2, init='uniform', activation='linear'),
    Layer(n_neurons=10, init='uniform', activation='linear')
)

In [608]:
model_ffnn.fit(X_train, y_train)

Training Progress:   0%|          | 0/2 [00:00<?, ?epoch/s]

Epoch 1


Training Progress:  50%|█████     | 1/2 [00:06<00:06,  6.39s/epoch]

Dur 6.387257814407349
Epoch 2


Training Progress: 100%|██████████| 2/2 [00:12<00:00,  6.22s/epoch]

Dur 6.053350448608398
Dur Global 12.443609952926636


In [609]:
model_ffnn.visualize()

In [610]:
for layer in model_ffnn.layers:
    print("=====Layer======")
    for i in layer.weights:
        print("==Neuron==")
        for j in i:
            print(j.varname)

=====Layer======
==Neuron==
out_add_8ff53888-ddd0-4952-b55b-cd4f31174cbd
out_add_484647eb-11c6-4717-a95d-2912625a4ff0
out_add_b8d1a867-e975-4df6-8ab0-45f34b0c5d84
out_add_25d3bd6b-9bc9-43d1-902b-07e354afd6e2
==Neuron==
out_add_e46103ee-666b-4684-a983-cde22d44c638
out_add_732434e0-eeed-41b3-995d-07dbdbeb0eb7
out_add_5d84b86e-249f-4c91-acdb-3dffd3c0d127
out_add_8072e0eb-e55f-4a0b-bdd1-a83b7a8bd85c
==Neuron==
out_add_fc1f9bb9-6573-4e6a-ac94-8f49cbbf4a82
out_add_e98d7d99-4f67-451c-9ccf-9c86e2fb3ae2
out_add_e27788d8-6a06-47b5-9005-8a36ea08deca
out_add_88b0f282-4f3a-4918-82a6-ef4f56208075
==Neuron==
out_add_5e459ed7-228d-47dc-bef4-a90aec639e30
out_add_807dd57f-c579-4a89-ad09-299f3970d960
out_add_c67ca4c8-a0f2-4019-be3d-3d8781b1cc06
out_add_4c7f0eee-6db0-4138-8f79-61cf5e5faeaf
==Neuron==
out_add_7b870c2e-5526-47b7-a07b-c263cd55ef39
out_add_02fe139c-dbbc-42a0-85b0-3d289600ac33
out_add_777d2f8c-b33a-465d-b07d-26c4bac1772c
out_add_09a7e97e-c58f-4eeb-90d5-f65dca7d4cc3
==Neuron==
out_add_ee235d11-

In [611]:
for layer in model_ffnn.layers:
    for i in layer.weights:
        for j in i:
            print(j.value)

-0.0259
-0.0234
-0.0352
0.0146
0.0204
-0.0063
0.0078
0.0013
-0.029
-0.0296
0.0235
-0.0047
0.0004
0.0283
-0.0189
-0.0227
0.0153
0.0264
-0.0101
0.0342
-0.025
-0.0079
0.0309
-0.0053
0.0246
-0.0081
0.005
0.0192
0.0343
-0.0247
-0.0259
-0.0323
0.0233
0.0046
0.0249
-0.0257
-0.0062
0.0131
0.0215
0.0122
-0.0104
0.0351
-0.029
0.0189
-0.006
0.0066
-0.0254
-0.0249
-0.0173
0.0186
-0.0353
0.0286
-0.0237
0.0258
0.03
0.0272
-0.0122
-0.0315
0.0192
0.027
-0.0198
-0.0137
-0.0137
0.0304
-0.0263
-0.0202
0.0239
0.0186
0.0078
0.0197
-0.0223
0.0169
0.0082
0.0272
-0.0073
-0.0327
0.0272
-0.0052
0.0129
0.0001
-0.0257
0.0207
-0.0258
0.0216
-0.0018
0.0201
0.0265
-0.0259
0.0177
0.021
-0.0173
0.0339
-0.0112
0.0033
-0.0173
0.0282
0.0127
0.0157
0.0316
0.0124
0.024
0.0171
-0.0204
-0.0053
0.0234
0.0179
-0.0204
-0.028
-0.0015
-0.0187
0.0233
-0.0225
0.0314
0.0301
-0.0213
0.033
-0.0294
0.0203
0.0168
-0.0235
0.0217
0.0192
-0.0216
0.0116
-0.0097
0.0214
0.0152
0.0003
-0.0185
0.0136
0.0029
-0.0313
0.0204
-0.0007
0.0049
-0.0116

In [612]:
y_pred = model_ffnn.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

5
5
5
5
5
5
5
5
5
5


In [613]:
model_ffnn.y.shape

(10, 10)

In [614]:
for h in y_test[:10].flatten():
    print(h)

9
6
5
9
6
9
8
8
1
2


In [615]:
model_ffnn.save(filename="try1.pth")

Model saved to try1.pth


In [616]:
new_model = FFNN()
new_model.load(filename="try1.pth")

Model loaded from try1.pth


In [617]:
y_pred = new_model.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

5
5
5
5
5
5
5
5
5
5
